# Initialise the SympNet

We will use symplectic neural networks with quadratic ridge polynomials (which is the best method for quadratic Hamiltonians) by setting the optional arguments `max_degree=2` and `method='P'`.

In [6]:
# add parent path to sys.path to import utils
from strupnet import VolNet, SympNet
import torch

DIM = 3  # dimension of the ODE

sympnet_kwargs = dict(
    layers=6,
    width=8,
    method="R",
)

volnet = VolNet(dim=DIM, **sympnet_kwargs)

# Generate data 

We will generate data of the form $ \{x(ih)\}_{i=0}^{n+1}$, where $\psi_h(x(0))=x(h)$ is a volume preserving map with unit determinant. The data is arranged in the form $ x_0 = \{x(ih)\}_{i=0}^{n} $, $ x_1 = \{x((i+1)h)\}_{i=0}^{n} $ and same for $ t $. 


In [7]:
torch.manual_seed(0)


def div_free_ode(x):
    """Create div-free ODE where d/dx_i(f_i)=0"""
    return (
        torch.sin(x[..., 1] * x[..., 2]),
        torch.cos(-x[..., 0] * x[..., 2]),
        torch.cos(-x[..., 0] * x[..., 1]),
    )


def volume_preserving_map(x_in, h, f=div_free_ode):
    """Create a volume preserving map using a splitting method"""
    x = x_in.clone()
    f0, _, _ = f(x)
    x[:, 0] = x[:, 0] + h * f0
    _, f1, _ = f(x)
    x[:, 1] = x[:, 1] + h * f1
    _, _, f2 = f(x)
    x[:, 2] = x[:, 2] + h * f2
    return x


def volume_preserving_data(timestep=0.05, n_points=100):
    x0 = torch.rand(n_points, DIM) * 2 - 1
    x1 = volume_preserving_map(x0, h=timestep)
    t0 = torch.zeros(n_points, 1)
    t1 = timestep * torch.ones(n_points, 1)
    return x0, x1, t0, t1

TIMESTEP = 0.1
x0, x1, t0, t1 = volume_preserving_data()

# Train the `VolNet` like any PyTorch model 
All the models in `strupnet` inherit from `torch.nn.Module` and can be trained as such. The loss function can be defined as follows. Letting $\Phi_h^{\theta}(x)$ denote the network, where $\theta$ denotes its set of trainable parameters, then we want to find $\theta$ that minimises 

$\qquad loss=\sum_{i=0}^{n}\|\Phi_h^{\theta}(x(ih))-x\left((i+1)h\right)\|^2$


In [8]:
optimizer = torch.optim.Adam(volnet.parameters(), lr=0.01)
mse = torch.nn.MSELoss()
for epoch in range(10000):
    optimizer.zero_grad()    
    x1_pred = volnet(x=x0, dt=t1 - t0)
    loss = mse(x1, x1_pred)
    print("Epoch: ", epoch, "Loss: ", loss.item()) if epoch % 1000 == 0 else None
    loss.backward()
    optimizer.step()

print("Final loss value: ", loss.item())

Epoch:  0 Loss:  0.001589333537069788
Epoch:  1000 Loss:  4.596445511723816e-06
Epoch:  2000 Loss:  2.447653168846114e-06
Epoch:  3000 Loss:  1.38057978441213e-06
Epoch:  4000 Loss:  8.227833700142502e-07
Epoch:  5000 Loss:  7.382183144652395e-07
Epoch:  6000 Loss:  6.936953094251066e-07
Epoch:  7000 Loss:  6.142210215917696e-07
Epoch:  8000 Loss:  5.638876662020627e-07
Epoch:  9000 Loss:  7.436061074065027e-07
Final loss value:  5.043130712668192e-07


# Evaluate the trained model on a test data set

In [9]:
x0, x1, t0, t1 = volume_preserving_data()
x1_pred = volnet(x=x0, dt=t1 - t0)
loss = mse(x1, x1_pred)
print("Final loss value on test data: ", loss.item())

Final loss value on test data:  1.7377077673535785e-06


# Check that the volnet has unit Jacobian determinant

In [10]:
# compute jacobians of volnet at the points x0 in test set
jacobians = torch.autograd.functional.jacobian(lambda x: volnet(x, 0.1), x0).sum(2).reshape(-1, DIM, DIM)
determinants = torch.det(jacobians)
mae = torch.mean(torch.abs(determinants - 1))
print("The determinant of the volnet evaluated on the test set has mean absolute error:\n\t", mae.item())

The determinant of the volnets evaluated on the test sets have mean absolute error:
	 2.2870594307278225e-16
